In [2]:
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from multiprocessing import Pool
import subprocess

In [6]:
#subprocess.Popen("aws s3 cp s3://jora-data-science/personalised-ctr-model/jora-search-clicks.csv000 jora-search-clicks.csv000".split())
df = pd.read_csv('../jora-search-clicks.csv000', sep = '\t', header=None, names=['user_id', 'job_title', 'click_ind'], nrows=10000).dropna()

In [7]:
df.user_id.value_counts()

750185                  10
2WiBn4vyjyyv9n-veuS8     7
eCy8PVzDVi9nu_csknHz     6
cFaGomySHntifrNT2Fv4     6
DC5heqieeJDYAzgEmFP9     5
DNzzeFH9Cn1LsBSYrVZx     5
wnD7jusPZQWaKZ2AzJVH     5
p8P-B_suxHsyedSt5XzK     4
NZW4YHxAeVsTUtC6w5xW     4
Ezdyds74RYGktVgCWXer     4
-YE2BNEvsR7brMsj5w2f     4
aTz3S_5goysVUWno89X_     4
CdG6daoz4nkUz__Z4us2     4
EzqxoDcSkEWPuWnT8zH9     4
ZaJX9VfSsyvMAqo1xdby     4
M-XUATtgz1M-xs25zBT6     4
Wr7FJ8gzmmy5oqsr4RYs     4
ziQkttGmxawbiUNMndBf     4
z1uUxHD5Reda4Qiwu7EE     3
C-Qy6jWzSE8sxG6Zvy2g     3
b1dwVJcS4ZAyutswcZJX     3
G3eWzy671crsp4fRE_nj     3
Lrna-iCXV2JJADYt6R_u     3
zKzswmx7-pqaeKf7hk6b     3
gofdv6hJxyuYoxyJLNc8     3
NTkXSAxy5rjYEEsdQDSz     3
Kp1DAwqDu3Cm9EM_MBsx     3
885918                   3
xNvoXYY7ZVcVT7yvpiyh     3
7p1fhwDYZkYzATzBZgMy     3
                        ..
vdNsZqZGd-ydsN1oBnzU     1
aczaRAxpbvDypE2TiqPt     1
WnvczTUtX-D4F35y2id7     1
TTsBs7-uj36YxnPhiSza     1
-oxV_8tkCFGDMiryMckX     1
be86B3EjjYzrb7TVfR9N     1
n

In [8]:
df[df['user_id'] == '2WiBn4vyjyyv9n-veuS8']

,user_id,job_title,click_ind
151,2WiBn4vyjyyv9n-veuS8,SUPPORT WORKER,1
1557,2WiBn4vyjyyv9n-veuS8,Support Assistant/Support Worker,1
3599,2WiBn4vyjyyv9n-veuS8,"PA / Carer Required – Private Employer £1,300 pcm",1
5555,2WiBn4vyjyyv9n-veuS8,Care Assistants Wanted!,1
6429,2WiBn4vyjyyv9n-veuS8,Autism and Challenging Behavior - Support Worker,1
6546,2WiBn4vyjyyv9n-veuS8,SEN Teaching Assistant - SEN TA - London,1
8217,2WiBn4vyjyyv9n-veuS8,Quality Coordinator,1


In [4]:
users = df.user_id.value_counts().ix[lambda x: x > 200].index

In [ ]:
def train_model(tup):
 try:
     user, group = tup
     X = group['job_title']
     y = group['click_ind']
     clf = make_pipeline(CountVectorizer(), LogisticRegression())
     clf.fit(X,y)
     coeffs = sorted(zip(clf.steps[0][1].get_feature_names(), clf.steps[1][1].coef_[0]), key = lambda t: abs(t[1]), reverse = True)
     query = " ".join("{}^{:4f}".format(k, v) for k, v in coeffs)
     return user + "\t"+ query
 except:
     return

pool = Pool()
res = pool.map(train_model, df[df.user_id.isin(users)].groupby('user_id'))
res

In [ ]:
with open('models.tsv', 'w') as f:
    f.write('\n'.join([r for r in res if r]))